<a href="https://colab.research.google.com/github/raymondwcs/learning_spacy/blob/main/iDDY_Dataset_Exploratory_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown --id 15qdMBG9IDtLlIc4WTHbwirvjot4ggtyF   # Google Drive file ID

Downloading...
From: https://drive.google.com/uc?id=15qdMBG9IDtLlIc4WTHbwirvjot4ggtyF
To: /content/threadPost15Column.tsv
50.1MB [00:00, 61.5MB/s]


In [2]:
import pandas as pd
from tqdm import tqdm

# Using readlines()
FILENAME='threadPost15Column.tsv'

file1 = open(FILENAME, 'r')
Lines = file1.readlines()
 
count = 0

tid = []
timestamp = []
subject = []
msg_01 = []
msg_02 = []
msg_03 = []
msg_04 = []
msg_05 = []
msg_06 = []
label_01 = []
label_02 = []
label_03 = []
label_04 = []
label_05 = []
label_06 = []

# Strips the newline character
for line in tqdm(Lines):
  count += 1
  cols = line.split("\t\t")
  if (len(cols) != 15):
    print("Line{}: cols={} {}".format(count, len(cols), line.strip()))
  else:
    # exclude message-less & label-less threads
    if cols[9] == 'neu' or cols[9] == 'pos' or cols[9] == 'neg':   
      tid.append(cols[0])
      timestamp.append(cols[1])
      subject.append(cols[2])
      msg_01.append(cols[3])
      msg_02.append(cols[4])
      msg_03.append(cols[5])
      msg_04.append(cols[6])
      msg_05.append(cols[7])
      msg_06.append(cols[8])
      label_01.append(cols[9])
      label_02.append(cols[10])
      label_03.append(cols[11])
      label_04.append(cols[12])
      label_05.append(cols[13])
      label_06.append(cols[14].strip())   # remove \n

print('Read {}, {} lines'.format(FILENAME,count))

100%|██████████| 215845/215845 [00:00<00:00, 417486.15it/s]

Read threadPost15Column.tsv, 215845 lines


Number of threads (excluded **message-less**, **label-less** threads)

In [3]:
# Create a dataframe from filtered raw text
df = pd.DataFrame({
    'tid': tid,
    'timestamp': timestamp,
    'subject': subject,
    'msg_01': msg_01,
    'msg_02': msg_02,
    'msg_03': msg_03,
    'msg_04': msg_04,
    'msg_05': msg_05,
    'msg_06': msg_06,
    'label_01': label_01,
    'label_02': label_02,
    'label_03': label_03,
    'label_04': label_04,
    'label_05': label_05,
    'label_06': label_06
})
print('dataframe: {} rows, {} columns'.format(len(df),len(df.columns)))

dataframe: 58194 rows, 15 columns


In [4]:
df.head(10)

,tid,timestamp,subject,msg_01,msg_02,msg_03,msg_04,msg_05,msg_06,label_01,label_02,label_03,label_04,label_05,label_06
0,DISCU_57_29839462,1619013446,華為無人駕駛系統再帶領A股進入大牛市,smile_o05.gif,cow,buy,升,,,neu,,,,,
1,DISCU_57_29839355,1619010602,夜期已經開始.........,"下跌乏力,明天大爆",天日284二八八兩邊發 收八六,<img src=,,,,neu,neu,neu,,,
2,DISCU_57_29839333,1619009841,跌市已經完結...........…,"終於安心晒,跌500點果陣已入牛再出發",聽日又跌,即係牛安心上路?🤔,好快五窮月,,,neu,neu,neu,,,
3,DISCU_57_29838788,1618993010,京東9618 vs 美團3690,現價: 京東 $287.20(PE 14.74) 美團$282.20(PE 292.30) ...,3690,老千股,,,,neu,neu,neu,,,
4,DISCU_57_29838728,1618991023,美團(3690)被大戶輪流洗倉了,美團(3690)被大戶輪流洗倉了,,,,,,neu,,,,,
5,DISCU_57_29838700,1618990508,588.5騰訊！快D！！,等左好耐。,<img src=,,,,,neu,neu,,,,
6,DISCU_57_29838693,1618990301,大市缺乏經濟火車頭 印度SENSEX接大陸棒？,大陸股到哂頂啦 即是香港股到哂頂啦 整固期可以長達一年以上 - 買過CSI300...,,,,,,neu,,,,,
7,DISCU_57_29838625,1618987645,快手！ 歷史低啦！！ 快手啦！！！🎉🎉🎉,你今日快手d 啦 20210421_ea03f3dc74f222fd675fWcG8h9Sp...,招股價幾多 cry.gif cry.gif,115 smile_35.gif,韭菜1斤斤的抬上貨車🚚運走🤤,老千 cry.gif cry.gif,抖音上。佢就係咁先,neu,neu,neu,neu,neu,neu
8,DISCU_57_29838618,1618987319,【滙豐HSBC】滙豐多名倫敦高層改駐香港傳惹內部不滿 消息：揣測無事實根據,【滙豐HSBC】滙豐多名倫敦高層改駐香港傳惹內部不滿 消息:揣測無事實根據,總部幾時搬番黎先 smile_34.gif smile_34.gif smile_34....,這己表示香港市場比英國重要。,來自香港同亞洲利潤遠大於英國歐洲,,,neu,neu,neu,neu,,
9,DISCU_57_29838599,1618986286,調查指小米全球智能手機市佔率排第三 銷量增八成,,勿買。。。遲早好似華為咁被搞 smile_42.gif,,,,,neu,neu,,,,


In [5]:
df.loc[df['tid'] == 'DISCU_57_29837114']

,tid,timestamp,subject,msg_01,msg_02,msg_03,msg_04,msg_05,msg_06,label_01,label_02,label_03,label_04,label_05,label_06
52,DISCU_57_29837114,1618908481,9923移卡暴升10%,女股神愛股啟動 幾時到 閱文 眾安 平安好醫生 ????,請問9923睇幾多?,funk.gif funk.gif 又爆升,高位120蚊跌落黎 怕咩 吉利36蚊跌落黎 依家起緊 小米又係35蚊跌落黎 又起緊 ...,<img src=,今日又升5% .話咁快71蚊啦,neu,neu,neu,neu,neu,


In [6]:
df.loc[df['tid'] == 'DISCU_57_29756443']

,tid,timestamp,subject,msg_01,msg_02,msg_03,msg_04,msg_05,msg_06,label_01,label_02,label_03,label_04,label_05,label_06
2379,DISCU_57_29756443,1615039463,星期五美股咁樣，咁星期一既港股如無意外.....,升完會否跌呢??,美股跌咗100點?,美國細價股洗倉,,,,neu,neu,neu,,,


Number of 'neu', 'pos' and 'neg' labels?

In [7]:
columns = ['label_01', 'label_02', 'label_03', 'label_04', 'label_05', 'label_06']
neu = 0
pos = 0
neg = 0

for col in columns:
  cneu = df[col].str.count('neu').sum()
  neu += cneu
  cpos = df[col].str.count('pos').sum()
  pos += cpos
  cneg = df[col].str.count('neg').sum()
  neg += cneg
  print('{}, {}, neu: {}, pos: {}, neg: {}'.format(col,pd.unique(df[col]),cneu,cpos,cneg))

print()
print('neu: {}, pos: {}, neg: {}'.format(neu, pos, neg))

label_01, ['neu' 'pos' 'neg'], neu: 44589, pos: 8003, neg: 5602
label_02, ['' 'neu' 'neg' 'pos'], neu: 37952, pos: 5693, neg: 5426
label_03, ['' 'neu' 'pos' 'neg'], neu: 33560, pos: 4908, neg: 4809
label_04, ['' 'neu' 'neg' 'pos'], neu: 30007, pos: 4367, neg: 4094
label_05, ['' 'neu' 'neg' 'pos'], neu: 26959, pos: 3964, neg: 3717
label_06, ['' 'neu' 'neg' 'pos'], neu: 24463, pos: 3484, neg: 3364

neu: 197530, pos: 30419, neg: 27012


How many messages are there per thread (subject)?



In [8]:
columns = ['label_01', 'label_02', 'label_03', 'label_04', 'label_05', 'label_06']
msg = 0
for index, row in df.iterrows():
  msg_per_subject = 0
  for col in columns:
    if row[col] == 'neu' or row[col] == 'pos' or row[col] == 'neg':
      msg_per_subject += 1
  msg += msg_per_subject

print('{:.1f}'.format(msg/len(df)))

4.4


Average message length

In [9]:
columns = ['subject', 'msg_01', 'msg_02', 'msg_03', 'msg_04', 'msg_05', 'msg_06']

mean_msg_length = []
for col in columns:
  mean_msg_length.append(df[col].apply(len).mean())

# print(mean_msg_length)

for col, avg_len in zip(columns,mean_msg_length):
  print('{}: {:.1f}'.format(col,avg_len))

subject: 18.3
msg_01: 84.3
msg_02: 24.8
msg_03: 20.9
msg_04: 18.7
msg_05: 16.8
msg_06: 15.2


Find most frequent tokens, nouns, verbs, adjectives and named entities

In [10]:
!pip install -U spacy
!python -m spacy download zh_core_web_lg
!git clone https://github.com/raymondwcs/learning_spacy

     |████████████████████████████████| 5.8 MB 6.3 MB/s 
     |████████████████████████████████| 10.1 MB 23.5 MB/s 
     |████████████████████████████████| 456 kB 47.8 MB/s 
     |████████████████████████████████| 42 kB 1.9 MB/s 
     |████████████████████████████████| 623 kB 69.8 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 603.8 MB 8.8 kB/s 
     |████████████████████████████████| 2.4 M

In [11]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-27807623-e4b3-6628-513a-d040c30916e9)


In [12]:
import spacy

tk = []       # tokens
noun = []     # nouns, proper nouns
verb = []     # verbs
adj = []      # adjectives
ne = []       # named entities

print(spacy.prefer_gpu()) # or spacy.require_gpu()
nlp = spacy.load('zh_core_web_lg')
# Use trained pkuseg model on Traditional Chinese text
# Download folder "models" from:
# https://github.com/raymondwcs/learning_spacy/tree/main/spacy_pkuseg
nlp.tokenizer.initialize(pkuseg_model='./learning_spacy/spacy_pkuseg/models')

from spacy.lang.zh.stop_words import STOP_WORDS
spacy_stopwords = STOP_WORDS
for w in spacy_stopwords:
  nlp.vocab[w].is_stop = True

columns = ['subject','msg_01','msg_02','msg_03','msg_04','msg_05','msg_06']
for index, row in tqdm(df.iterrows(), leave=True):
  for col in columns:
    msg = row[col].replace(" ","")
    if (len(msg) == 0):
      continue
    doc = nlp(msg)
    # remove stop words
    tokens = [token for token in doc if not token.is_stop]
    # for token in doc:
    for token in tokens:
        tk.append(token.text)
        if token.pos_ == 'NOUN' or token.pos_ == 'PROPN':
            noun.append(token.text)
        elif token.pos_ == 'VERB':
            verb.append(token.text)
        elif token.pos_ == 'ADJ':
            adj.append(token.text)
    for ent in doc.ents:
        ne.append('(%s, %s)' % (ent.text, ent.label_))

True


58194it [1:03:51, 15.19it/s]


In [13]:
pd.set_option('display.max_rows', None) 

In [14]:
pd.Series(tk).value_counts().head(3000)

係                                          26804
個                                          22862
會                                          19169
點                                          19157
買                                          18672
                                           17879
升                                          17557
元                                          16429
跌                                          15677
港股                                         12155
中國                                         11859
香港                                         11707
幾                                          10887
今日                                         10765
公司                                         10437
年                                          10051
佢                                           9582
「                                           9362
過                                           9301
將                                           8870
為                   

In [15]:
pd.Series(noun).value_counts().head(3000)

港股                                                  12155
中國                                                  11859
香港                                                  11707
今日                                                  10765
公司                                                  10437
美國                                                   8417
騰訊                                                   8163
股市                                                   7904
市場                                                   7236
股票                                                   5680
小米                                                   5406
股價                                                   5245
錢                                                    4958
月                                                    4725
美股                                                   4637
大戶                                                   4566
指數                                                   4449
銀行            

In [16]:
pd.Series(adj).value_counts().head(3000)

新                                                   4401
高                                                   1847
恆指                                                  1276
低                                                    942
相關                                                   854
內地                                                   789
個                                                    767
前                                                    724
壞                                                    699
未來                                                   692
沽空                                                   655
這個                                                   601
有關                                                   535
無                                                    473
最低                                                   457
賭變                                                   440
大型                                                   423
幾多                             

In [17]:
pd.Series(verb).value_counts().head(3000)

係                                                   24860
買                                                   18221
升                                                   16852
跌                                                   15625
為                                                    7710
做                                                    6495
過                                                    5841
想                                                    5755
指                                                    5684
睇                                                    5507
走                                                    4269
入                                                    4221
無                                                    4095
玩                                                    3847
見                                                    3770
賺                                                    3678
講                                                    3561
會             

In [18]:
pd.Series(ne).value_counts().head(3000)

(香港, GPE)                                           9750
(港股, GPE)                                           7052
(今日, DATE)                                          6778
(美國, GPE)                                           6554
(中國, GPE)                                           6079
(騰訊, ORG)                                           5608
(三, CARDINAL)                                       3448
(小米, ORG)                                           3109
(明天, DATE)                                          3009
(一, CARDINAL)                                       2959
(今年, DATE)                                          2954
(3, CARDINAL)                                       2893
(美股, GPE)                                           2723
(2, CARDINAL)                                       2582
(700, CARDINAL)                                     2472
(1, CARDINAL)                                       2344
(第一, ORDINAL)                                       2238
(大陸, GPE)                      